## Background

Given the need to examine family-based vs population-based design, testing for genetic association with rare variants (allele) is crucial for genome wide association studies (GWAS). However, systemic differences between allele frequencies in subpopulations   remains a confounding variable  in case-control studies, where association could be found not by a disease associated marker but rather the underlying population structure. Employing a variant-set mixed model framework to allow for fixed and random effects, we are able to properly analyze  complex hierarchical data while accounting for population stratification and cryptic association. In order to mitigate this risk, we utilize a genome wide mixed model association test (GEMMA) to correct population structure and sample non-exchangeability and test multiple phenotypes. 


## Project Objective

The goal of this family based association study design is to utilize stimulation studies to analyze a family-based vs. population based design. Power is gained in analyzing families due to the effect variants that should be enrished within the family. However, correlation between family member limits the sample size of the study. Also, incomparison to population controls, controls within families allow them to be likely carriers of disease alleles which can decrease power. 

We therefore look into different strategies to assess the trade-off:

1) Analyze families using LMM including both affected and unaffected individuals. We can either analyze all families, or analyze them separately and combine the outcome later.
2) Analyze unrelated case control design of the same sample size and phenotypic model, and see how much larger the effect size have to be in families (in step 1), to offset the loss of power due to using related individuals, compared to unrelated design.
3) Analyze affected individuals in the families but replaced the unaffected members with related controls from other families of the same structure, where no one is affected in their family. That is, the controls are still related, but perhaps no longer carrying a risk allele.
4) Analyze affected family members and replaced the unaffected members with population-based controls. Here the controls are unrelated.
5) Analyze a sample of unrelated individuals with the same number of affected and unaffected individuals as in analysis 1-3.

## Replication Objective 

The goal of the replication is to verify the effect size of the association, compare relatedness of families, and ultimately confirm the genetic relatedness matrix results of the stimulated genetic relationship matrix (GRM), created by Joe Chase. 

This file contains the percent of genetic material shared identically by descent by each selected individual in order to properly assess population stratification and cryptic relatedness. Utilizing known information on ancestor pedigrees, an estimated genetic relationship matrix is highly comparable for assessing phenotype similarity. Through understanding how this file was generated and replicating the results, we can assert the validity of the results and use the GRM as a base for future family-based association study design objectives where effect variants are enriched within families while maintaining a substantial sample size to work with.

We first simulate genetic data and pedigree with phenotypes for the creation of families of size eight (two parents, six children). Since the original data is in haplotype form, the data is summed and appropriately replaced with either a 0, 1, or 2 to match the respective genotype format. To mitigate population stratification, we employ the SMMATs framework for effective random fixed effects. This ensured the creation of effect variant families with non-identical siblings through ensuring optimal recombination rates and a high migration parameter between generated populations. In order to replicate this genotype on a large scale for approximately 12,000 individuals, we use to create a .bed, .bim. and .fam files to feed into PLINK for large scale haplotype reproduction for the individuals. 

Families are created by assigning a parent's genome to each child while accounting for non-identical siblings through random sampling. In each subpopulation structure, the 20 founders' haplotypes are subsetted for each child creation, by which a new data frame of the complete family is created with each individual assigned a unique ID. To keep family size closer to reality, families are structured and filtered using census data on family household size. 

Selected SNPs are assigned a specific non-zero effect size to assign phenotype values which are selected based on SNP value and assigned random familial and individual noise. We then generate both the continuous and binary phenotypes associated with each individual for the final selection of affected families with >= 2 individuals affected.  


## Methods of verification
In order to verify these family creations for futher use in analyzing the different study designs, we must first check with multiple metrics to ensure genetic relationship matrix is accurate. 

The following metrics to verify our stimuation are used:

1) SNP Pairwise Estimation: Create a genotype matrix for selected snps to check for correlation
- Run Pairwise estimation for snp square matrix to ensure SNPs are not correlated with each other and so therefore the estimated effeect size can be accepted. 
- SNPs should have a diagnol correlation with itself on the matrix, that is a direct correlation of 1, while all other SNPs should have a correlation of 0.

2) Verify effect size for SNPs
- Compare 10 selected SNPs with no correlation to each other and verify effect size is desired effet size.

3) Compare Family Estimated Kingship Coefficient with pedigree matrix truth
- Using PLINK and KING, verify population percentages of estimates pair-wise kinship coefficients

4) GWAS Manhattan and Quantile-Quantile (QQ) plot
- Compare effect size of SNPs and verify significant P values 
  


### Method Verification: 1) SNP Pairwise Estimation: Create a genotype matrix for selected snps to check for correlation

In population-based and family genetics, linkage disequilibrium (LD) refers to the non-random association of alleles at different loci  in the population of interest. When loci are in linkage disequilibrium the frequency of association of their haplotypes is higher or lower than the expected value if loci were independent and associated randomly. Linkage disequilibrium can be influenced by a multitude of factors, including gene recombination rate, and population stratification. Therefore, population linkage disequilibrium outlines serve to identify the genetic processes responsible for its structure.

 The linkage disequilibrium score (LD) serves to quantify the separate contributions of multi gene inheritance effects and various confounding factors, such as population stratification or cryptic relatedness in GWAS studies. The score is based on a linear regression analysis examining the sum of the coefficient of determination, otherwise known as the R2, between each SNP with all other SNPs in the study. 

We assess LD in our simulation by assessing the LD score for the 100 snps assigned non-zero log ratio effects (log 1.5 for 60 and log 0.5 for 40). We stimulate this process via PLINK’s --ld-snp-list <filename> command and feed in the SNPs effect file. By filtering out selected SNPs with non-zero effects, we have a file containing the R2 for only the selected SNPs.

In [4]:
install.packages("magrittr")
install.packages("dplyr")    
library(magrittr)
library(dplyr)   
library(data.table)
library(tidyverse)

f_name <- "new_six_fam_eff"

setwd("C:/User/samanthafigueredo/output") 
print(getwd())

ld<-fread(paste0("output/", f_name, ".ld")) %>% as_tibble()
sel_snp_corr <- ld %>% filter(SNP_A %in% snps) %>% filter(SNP_B %in% snps)
  
write.table(sel_snp_corr, file = "output/sel_snp_cor.txt", 
              sep = "\t", col.names = T, 
              row.names = F,
              quote = F)



The downloaded binary packages are in
	/var/folders/gl/7vjrd5hx6rs6y0v55jzsw38c0000gn/T//RtmpaSBUTv/downloaded_packages

The downloaded binary packages are in
	/var/folders/gl/7vjrd5hx6rs6y0v55jzsw38c0000gn/T//RtmpaSBUTv/downloaded_packages


ERROR: Error in setwd("C:/User/samanthafigueredo/output"): cannot change working directory


In [7]:
sel_snp_corr_check<- fread("output/sel_snp_cor.txt")

ERROR: Error in fread("output/sel_snp_cor.txt"): File 'output/sel_snp_cor.txt' does not exist or is non-readable. getwd()=='/Users/samanthafigueredo/output'


We then create a square genotype matrix for the 100 snps to assess LD pairwise estimation for correlation. 

In [ ]:
install.packages("ggcorrplot")
library(ggcorrplot)

# creating square SNP matrix 
valnames<-unique(sel_snp_corr$SNP_A)
myMat <- matrix(0, length(valnames), length(valnames), dimnames = list(valnames, valnames))
myMat[as.matrix(sel_snp_corr[c("SNP_A", "SNP_B")])] <- sel_snp_corr[["R2"]]

corr <- cor(myMat)
ggcorrplot(corr, title = "Correlation of Selected SNPs", legend.title = "Correlation", tl.cex = 4)

ggcorrplot(corr) + 
  ggplot2::labs(x = 'SNPs', y = 'SNPs')




As can be seen by the correlation_of_snp_graph.pdf, each SNP is in perfect correlation with itself while having 0 correlation with other SNPs. Only a few SNPs have high LD. This pattern allows us to further verify and estimate the true effect size of each SNP not in high LD with each other. From this visual, we are now able to assess whether the SNP effect size is the desired effect size after noise is applied and as well as determine which snps have an inflated effect size, which we can attribute to high LD. 

### 2) Verify effect size for SNPs

After verifying the linear correlation for the selected non-zero effect SNPs, we can now verify the effect size of each SNP after applying noise. There are two types of noises affecting the effect size of each SNP: error associated with each individual (epsilon) in the creation of the phenotype function and familial error which has a multivariable normal distribution with a covariate matrix for each family. This causes family grouping to have error in the same direction and presumably by a similar magnitude. 


In [ ]:
#Loading in file with all SNP effect size
assoc<- fread("output/new_six_fam_eff.assoc.txt")

# Filter and save 100 non-zero effect size snps and 
eff_comparison <- assoc %>% filter(rs %in% snps)

# Filter by SNP ID and effect size only (beta)
eff_comparison<- subset(eff_comparison, select=c("rs","beta"))

# Add side by side comparison of actual effect size before added noise
eff_comparison$effect_size <- my_snps$V2
#View(eff_comparison)

# save 
write.table(eff_comparison, paste0("output/size_eff_comparison.txt"), sep="\t", quote=F, 
            row.names = F, col.names = T)

This outfile contains all selected SNPs correlation with each other SNP. We select SNPs for verification of effect size by filtering out 10 SNPs with an association of less than 10%

In [ ]:
# Select 10 SNPS NOT in correlation with one alother. These SNPs have a correlation
# with any other SNP larger than 10%, based on R2 value
eff_comparison$abs_diff <- abs(eff_comparison$beta - eff_comparison$effect_size)
View(eff_comparison)

ten_snp_effect_size <- eff_comparison[c(28,29,30,31,32,33,34,35,36,37)]
View(ten_snp_effect_size)

write.table(ten_snp_effect_size, paste0("output/ten_snp_size_eff_comparison.txt"), sep="\t", quote=F, 
            row.names = F, col.names = T)

Next, we calculate the 95% confidence intervals for each each selected snp's effect size compared to their true effect size. Because these SNPs are independently correlated, we can assume their effect size to be closer to the true effect size after accounding for error.

In [ ]:
########### Calculating 95% CI for each selected ten SNP effect size #########
# snp_1 beta value and true size
snp_1<- c(sel_snps[1,])
snp_1_eff<- c(snp_1[[1]],snp_1[[2]])
# calculate mean 
mean_1<- mean(snp_1_eff)
# calculate sd 
sd_1<-sd(snp_1_eff)
# calculate error 
error_1 <- qnorm(0.975) * sd_1/sqrt(2)
# calculate confidence level 
ub_1<- snp_1_mean+error_1
lb_1<- snp_1_mean-error_1

# snp_2 beta value and true size
snp_2<- c(sel_snps[2,])
print(snp_2)
snp_2_eff<- c(snp_2[[1]],snp_2[[2]])
# calculate mean 
mean_2<- mean(snp_2_eff)
# calculate sd 
sd_2<-sd(snp_2_eff)
# calculate error 
error_2 <- qnorm(0.975) * sd_2/sqrt(2)
# calculate confidence level 
ub_2<- mean_2+error_2
lb_2<- mean_2-error_2

# snp_3 beta value and true size
snp_3<- c(sel_snps[3,])
snp_3_eff<- c(snp_3[[1]],snp_3[[2]])
# calculate mean 
mean_3<- mean(snp_3_eff)
# calculate sd 
sd_3<-sd(snp_3_eff)
# calculate error 
error_3 <- qnorm(0.975) * sd_3/sqrt(2)
# calculate confidence level 
ub_3<- mean_3+error_3
lb_3<- mean_3-error_3

# snp_4 beta value and true size
snp_4<- c(sel_snps[4,])
snp_4_eff<- c(snp_4[[1]],snp_4[[2]])
# calculate mean 
mean_4<- mean(snp_4_eff)
# calculate sd 
sd_4<-sd(snp_4_eff)
# calculate error 
error_4 <- qnorm(0.975) * sd_4/sqrt(2)
# calculate confidence level 
ub_4<- mean_4+error_4
lb_4<- mean_4-error_4

# snp_5 beta value and true size
snp_5<- c(sel_snps[5,])
snp_5_eff<- c(snp_5[[1]],snp_5[[2]])
# calculate mean 
mean_5<- mean(snp_5_eff)
# calculate sd 
sd_5<-sd(snp_5_eff)
# calculate error 
error_5 <- qnorm(0.975) * sd_5/sqrt(2)
# calculate confidence level 
ub_5<- mean_5+error_5
lb_5<- mean_5-error_5

# snp_6 beta value and true size
snp_6<- c(sel_snps[6,])
snp_6_eff<- c(snp_6[[1]],snp_6[[2]])
# calculate mean 
mean_6<- mean(snp_6_eff)
# calculate sd 
sd_6<-sd(snp_6_eff)
# calculate error 
error_6 <- qnorm(0.975) * sd_6/sqrt(2)
# calculate confidence level 
ub_6<- mean_6+error_6
lb_6<- mean_6-error_6

# snp_7 beta value and true size
snp_7<- c(sel_snps[7,])
snp_7_eff<- c(snp_7[[1]],snp_7[[2]])
# calculate mean 
mean_7<- mean(snp_7_eff)
# calculate sd 
sd_7<-sd(snp_7_eff)
# calculate error 
error_7 <- qnorm(0.975) * sd_7/sqrt(2)
# calculate confidence level 
ub_7<- mean_7+error_7
lb_7<- mean_7-error_7

# snp_8 beta value and true size
snp_8<- c(sel_snps[8,])
snp_8_eff<- c(snp_8[[1]],snp_8[[2]])
# calculate mean 
mean_8<- mean(snp_8_eff)
# calculate sd 
sd_8<-sd(snp_8_eff)
# calculate error 
error_8 <- qnorm(0.975) * sd_8/sqrt(2)
# calculate confidence level 
ub_8<- mean_8+error_8
lb_8<- mean_8-error_8

# snp_7 beta value and true size
snp_9<- c(sel_snps[9,])
snp_9_eff<- c(snp_9[[1]],snp_9[[2]])
# calculate mean 
mean_9<- mean(snp_9_eff)
# calculate sd 
sd_9<-sd(snp_9_eff)
# calculate error 
error_9 <- qnorm(0.975) * sd_9/sqrt(2)
# calculate confidence level 
ub_9<- mean_9+error_9
lb_9<- mean_9-error_9


# snp_10 beta value and true size
snp_10<- c(sel_snps[10,])
snp_10_eff<- c(snp_10[[1]],snp_10[[2]])
# calculate mean 
mean_10<- mean(snp_10_eff)
# calculate sd 
sd_10<-sd(snp_10_eff)
# calculate error 
error_10 <- qnorm(0.975) * sd_10/sqrt(2)
# calculate confidence level 
ub_10<- mean_10+error_10
lb_10<- mean_10-error_10

# adding upper bound and lower bound columns to sel_snp data frame

ten_snp_effect_size$upper_bound<- cbind(ub_1,ub_2,ub_3,ub_4,ub_5,ub_6,ub_7,ub_8,ub_9,ub_10)
ten_snp_effect_size$lower_bound<- cbind(lb_1,lb_2,lb_3,lb_4,lb_5,lb_6,lb_7,lb_8,lb_9,lb_10)

Next, we plot our confidence intervals for the SNP log odds ratio estimate (beta). The dashed red nigh signifies the true log odds ratio before noise is applied. 

In [ ]:
# plotting confidence intervals for selected snps effect size 

ggplot(ten_snp_effect_size, aes(x=rs, y=beta))+
  ggtitle("95% Confidence Interval for 10 selected SNPs effect size")+
  geom_hline(aes(yintercept=0.4054651), linetype="dashed", color = "red", lty="boo")+
  scale_linetype_manual(name = "True effect size")+
  geom_errorbar(aes(ymin=lower_bound,ymax=upper_bound))+
  labs(y= "Log Odds Ratio Estimate", x = "RS ID", size= 19)+
  geom_point()


As can be seen, all SNP beta values fall between the calculated upper and lower bounds. 